Code based on
    analyse_SR_results.ipynb
by 
    Fabrizio Frasca
in commit 
    bc07009fc949a32313991090a46e0f932689d7bc
which was commited
    May 15th 2021
in repository  
    twitter-research/cwn 

In [57]:
import sys
sys.path.append('./../')
import os
from definitions import ROOT_DIR
import matplotlib.pyplot as plt
import glob
import numpy as np
from pprint import pprint

In [75]:
def print_results(path, args):
    with open(path, 'r') as handle:
        return handle.read()

def parse_results(path, args):
        with open(path, 'r') as handle:
            best_val_epoch_val = None
            best_val_epoch_test = None
            final_epoch_val = None
            final_epoch_test = None

            args_dict = None

            first_valid = True
            first_test = True
            for line in handle:
                if line.startswith('Valid'):
                    # if first_valid, we view the first line that starts with 'Valid',
                    # as that is the line that shows the val acc at the best epoch.
                    # else, we view the second line that starts with 'Valid', as that
                    # is the line that shows the val acc at the last epoch.
                    if first_valid:
                        best_val_epoch_val = line.strip().split(':                  ')[1]
                        best_val_epoch_val = float(best_val_epoch_val.split(' ± ')[0])
                        first_valid = False
                    else:
                        final_epoch_val = line.strip().split(':                  ')[1]
                        final_epoch_val = float(final_epoch_val.split(' ± ')[0])
                if line.startswith('Test:'):
                    if first_test:
                        best_val_epoch_test = line.strip().split(':                   ')[1]
                        best_val_epoch_test = float(best_val_epoch_test.split(' ± ')[0])
                        first_test = False
                    else:
                        final_epoch_test = line.strip().split(':                   ')[1]
                        final_epoch_test = float(final_epoch_test.split(' ± ')[0])
                elif line.startswith('Namespace'):
                    args_dict = {}
                    fields = line.lstrip('Namespace(').rstrip(')').strip().split(', ')
                    for field in fields:
                        try:
                            key, value = field.split('=')
                            if key in args:
                                args_dict[key] = value
                        except ValueError:
                            pass
                else:
                    pass
        return best_val_epoch_val, final_epoch_val, best_val_epoch_test, final_epoch_test, args_dict

def get_results(result_path, args, which_epoch='best'):
    results = list()
    for i in range(300):
        try:
            path = os.path.join(result_path, f'MOLHIV-{i}', 'result.txt')
            bval_val, fepo_val, bval_test, fepo_test, hypparam_settings = parse_results(path, args)
            if which_epoch=='best_val':
                results.append((i, bval_val, hypparam_settings))
            elif which_epoch=='final_val':
                results.append((i, fepo_val, hypparam_settings))
            elif which_epoch=='best_val_test':
                results.append((i, bval_test, hypparam_settings))
            elif which_epoch=='final_test':
                results.append((i, fepo_test, hypparam_settings))
        except FileNotFoundError:
            pass

    results.sort(reverse=True, key=lambda r:r[1])

    return results

# Preliminary tests

These are preliminary because after I ran them I realized
- I need to run over more repeats to ensure low variance
- Some hyperparameters don't matter as much as others
- I was accidentally tuning over final validation accuracy rather than best validation accuracy epoch's validation accuracy.

We sort by best validation performance epoch's validation performance

#### CIN Less Sparse

Results were not repeated (with multiple seeds), hence performance metric has high variance. Which is why the highest one is so high (0.869)

In [73]:
result_path = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_less_sparse')

args = ['drop_rate',
        'lr', 
        'lr_scheduler',
        'lr_scheduler_decay_rate',
        'omit_2cell_down',
        'num_layers',
        ]

res = get_results(result_path, args, which_epoch='best_val')
pprint(len(res))
pprint(res)

300
[(259,
  0.869353811483441,
  {'drop_rate': '0.5',
   'lr': '0.001',
   'lr_scheduler': "'None'",
   'lr_scheduler_decay_rate': '0.5',
   'num_layers': '3',
   'omit_2cell_down': "'True'"}),
 (206,
  0.8659060846560848,
  {'drop_rate': '0.5',
   'lr': '0.0001',
   'lr_scheduler': "'None'",
   'lr_scheduler_decay_rate': '0.9',
   'num_layers': '3',
   'omit_2cell_down': "'False'"}),
 (100,
  0.8622072800313542,
  {'drop_rate': '0.1',
   'lr': '0.001',
   'lr_scheduler': "'None'",
   'lr_scheduler_decay_rate': '0.5',
   'num_layers': '2',
   'omit_2cell_down': "'False'"}),
 (131,
  0.8615030374289633,
  {'drop_rate': '0.1',
   'lr': '0.003',
   'lr_scheduler': "'None'",
   'lr_scheduler_decay_rate': '0.5',
   'num_layers': '3',
   'omit_2cell_down': "'True'"}),
 (133,
  0.8605936458945718,
  {'drop_rate': '0.1',
   'lr': '0.003',
   'lr_scheduler': "'None'",
   'lr_scheduler_decay_rate': '0.5',
   'num_layers': '2',
   'omit_2cell_down': "'False'"}),
 (156,
  0.8595036008230453,
  {'

#### CIN Less Sparse 2

In [60]:
result_path = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_less_sparse2')

args = [
        'lr', 
        'omit_2cell_down',
        'num_layers',
        ]

res = get_results(result_path, args, which_epoch='best_val')
pprint(len(res))
pprint(res)

40
[(34,
  0.8499779541446207,
  {'lr': '0.0003', 'num_layers': '6', 'omit_2cell_down': "'True'"}),
 (26,
  0.8478675227807172,
  {'lr': '0.0001', 'num_layers': '3', 'omit_2cell_down': "'False'"}),
 (27,
  0.8462949694624077,
  {'lr': '0.0001', 'num_layers': '4', 'omit_2cell_down': "'False'"}),
 (29,
  0.8462214832778104,
  {'lr': '0.0001', 'num_layers': '6', 'omit_2cell_down': "'False'"}),
 (36,
  0.8461536106212031,
  {'lr': '0.0003', 'num_layers': '3', 'omit_2cell_down': "'False'"}),
 (22,
  0.8452605493500557,
  {'lr': '0.0001', 'num_layers': '4', 'omit_2cell_down': "'True'"}),
 (39,
  0.8449354750473578,
  {'lr': '0.0003', 'num_layers': '6', 'omit_2cell_down': "'False'"}),
 (33,
  0.8449222067084721,
  {'lr': '0.0003', 'num_layers': '5', 'omit_2cell_down': "'True'"}),
 (38,
  0.8444833308837939,
  {'lr': '0.0003', 'num_layers': '5', 'omit_2cell_down': "'False'"}),
 (32,
  0.8438143004115227,
  {'lr': '0.0003', 'num_layers': '4', 'omit_2cell_down': "'True'"}),
 (37,
  0.84351116990

#### CIN Less Sparse 3+4

In [61]:

result_path3 = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_less_sparse3')
result_path4 = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_less_sparse4')

args = [ 
        'lr', 
        'omit_2cell_down',
        'num_layers',
        'use_down_attr'
        ]

res3 = get_results(result_path3, args, which_epoch='best_val')
res4 = get_results(result_path4, args, which_epoch='best_val')
res3 = [(3,i) for i in res3]
res4 = [(4,i) for i in res4]
res = res3 + res4
res.sort(reverse=True, key=lambda r:r[1][1])
pprint(len(res))
pprint(res)

4
[(4,
  (0,
   0.8330865422300608,
   {'lr': '3e-05',
    'num_layers': '4',
    'omit_2cell_down': "'False'",
    'use_down_attr': "'False'"})),
 (4,
  (1,
   0.83121090943236,
   {'lr': '3e-05',
    'num_layers': '5',
    'omit_2cell_down': "'False'",
    'use_down_attr': "'False'"})),
 (3,
  (0,
   0.8303212162780064,
   {'lr': '3e-05',
    'num_layers': '4',
    'omit_2cell_down': "'False'",
    'use_down_attr': "'True'"})),
 (3,
  (1,
   0.824148580900124,
   {'lr': '3e-05',
    'num_layers': '5',
    'omit_2cell_down': "'False'",
    'use_down_attr': "'True'"}))]


#### CIN Dense

In [62]:
result_path = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_dense')

args = [
        'lr', 
        'omit_2cell_down',
        'num_layers',
        ]

res = get_results(result_path, args, which_epoch='best_val')
pprint(len(res))
pprint(res)

40
[(36,
  0.847407978966621,
  {'lr': '0.0003', 'num_layers': '3', 'omit_2cell_down': "'False'"}),
 (28,
  0.8469798198771964,
  {'lr': '0.0001', 'num_layers': '5', 'omit_2cell_down': "'False'"}),
 (38,
  0.8454473267359068,
  {'lr': '0.0003', 'num_layers': '5', 'omit_2cell_down': "'False'"}),
 (30,
  0.8430051868998628,
  {'lr': '0.0003', 'num_layers': '2', 'omit_2cell_down': "'True'"}),
 (39,
  0.8421618818995361,
  {'lr': '0.0003', 'num_layers': '6', 'omit_2cell_down': "'False'"}),
 (37,
  0.8405630470638186,
  {'lr': '0.0003', 'num_layers': '4', 'omit_2cell_down': "'False'"}),
 (34,
  0.8402782880985041,
  {'lr': '0.0003', 'num_layers': '6', 'omit_2cell_down': "'True'"}),
 (16,
  0.8395847622313671,
  {'lr': '3e-05', 'num_layers': '3', 'omit_2cell_down': "'False'"}),
 (17,
  0.8394393208243517,
  {'lr': '3e-05', 'num_layers': '4', 'omit_2cell_down': "'False'"}),
 (31,
  0.8390974059376837,
  {'lr': '0.0003', 'num_layers': '3', 'omit_2cell_down': "'True'"}),
 (26,
  0.8389759496048

# Additional adjacencies in CIN on ogb-MOLHIV

We sort by best validation performance epoch's validation performance

#### sparse-CIN

In [74]:
result_path = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_sparse')

args = [
        'lr', 
        'num_layers',
        ]

res = get_results(result_path, args, which_epoch='best_val')
pprint(len(res))
pprint(res[:6])

path = os.path.join(result_path, f'MOLHIV-{9}', 'result.txt')
print_results(path, args)

25
[(9, 0.8477613250375596, {'lr': '0.001', 'num_layers': '6'}),
 (7, 0.8431305727023318, {'lr': '0.001', 'num_layers': '4'}),
 (12, 0.8428294834737735, {'lr': '0.003', 'num_layers': '4'}),
 (14, 0.841301379090731, {'lr': '0.003', 'num_layers': '6'}),
 (2, 0.8411870672480238, {'lr': '0.0003', 'num_layers': '4'}),
 (8, 0.8398676840420668, {'lr': '0.001', 'num_layers': '5'})]


"<_io.TextIOWrapper name='/home/sunjin/cwn/exp/results/MOLHIV_tuning_molhiv_sparse/MOLHIV-9/result.txt' mode='r' encoding='UTF-8'>"

#### less-sparse-CIN   
Was conducted over two grids (less_sparse5 + less_sparse6)

In [64]:
result_path5 = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_less_sparse5')
result_path6 = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_less_sparse6')

args = [
        'lr', 
        'num_layers',
        ]

res5 = get_results(result_path5, args, which_epoch='best_val')
res6 = get_results(result_path6, args, which_epoch='best_val')
res5 = [(5,i) for i in res5]
res6 = [(6,i) for i in res6]
res = res5 + res6
res.sort(reverse=True, key=lambda r:r[1][1])
pprint(len(res))
pprint(res[:6])

25
[(5, (1, 0.8541558478672676, {'lr': '0.0003', 'num_layers': '4'})),
 (5, (6, 0.8493357665425567, {'lr': '0.001', 'num_layers': '5'})),
 (5, (0, 0.848931388398981, {'lr': '0.0003', 'num_layers': '3'})),
 (5, (15, 0.8485310928212162, {'lr': '0.0001', 'num_layers': '6'})),
 (5, (2, 0.8485176203540401, {'lr': '0.0003', 'num_layers': '5'})),
 (5, (14, 0.8470540205108107, {'lr': '0.0001', 'num_layers': '5'}))]


#### dense-CIN
was conducted over two grids (molhiv_dense2 + molhiv_dense3)

In [65]:
result_path2 = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_dense2')
result_path3 = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_dense3')

args = [
        'lr', 
        'num_layers',
        ]


res2 = get_results(result_path2, args, which_epoch='best_val')
res3 = get_results(result_path3, args, which_epoch='best_val')
res2 = [(2,i) for i in res2]
res3 = [(3,i) for i in res3]
res = res2 + res3
res.sort(reverse=True, key=lambda r:r[1][1])
pprint(len(res))
pprint(res[:6])

25
[(2, (2, 0.8549103060291332, {'lr': '0.0003', 'num_layers': '5'})),
 (2, (3, 0.8498177134365406, {'lr': '0.0003', 'num_layers': '6'})),
 (2, (1, 0.8471019906590893, {'lr': '0.0003', 'num_layers': '4'})),
 (2, (7, 0.8461460578744529, {'lr': '0.001', 'num_layers': '6'})),
 (2, (14, 0.8456720719838002, {'lr': '0.0001', 'num_layers': '5'})),
 (3, (1, 0.8446295888039717, {'lr': '0.001', 'num_layers': '2'}))]


# Others

#### Basic Sparse

In [66]:
result_path = os.path.join(ROOT_DIR, 'exp', 'results', 'MOLHIV_tuning_molhiv_basic_sparse')

args = [
        'lr', 
        'num_layers',
        ]

res = get_results(result_path, args, which_epoch='best_val')
pprint(len(res))
pprint(res)

15
[(3, 0.8423947922790516, {'lr': '0.0003', 'num_layers': '5'}),
 (12, 0.8400042458684432, {'lr': '0.003', 'num_layers': '4'}),
 (13, 0.8378688598210204, {'lr': '0.003', 'num_layers': '5'}),
 (8, 0.8378027222548827, {'lr': '0.001', 'num_layers': '5'}),
 (9, 0.8377565892612189, {'lr': '0.001', 'num_layers': '6'}),
 (6, 0.8367651381540269, {'lr': '0.001', 'num_layers': '3'}),
 (7, 0.8364960970670846, {'lr': '0.001', 'num_layers': '4'}),
 (14, 0.8335033721993599, {'lr': '0.003', 'num_layers': '6'}),
 (5, 0.8332788310797571, {'lr': '0.001', 'num_layers': '2'}),
 (11, 0.8332592347638644, {'lr': '0.003', 'num_layers': '3'}),
 (2, 0.8325770380168529, {'lr': '0.0003', 'num_layers': '4'}),
 (4, 0.832006499444771, {'lr': '0.0003', 'num_layers': '6'}),
 (0, 0.8311032317590958, {'lr': '0.0003', 'num_layers': '2'}),
 (10, 0.8301056568031877, {'lr': '0.003', 'num_layers': '2'}),
 (1, 0.8294087628192567, {'lr': '0.0003', 'num_layers': '3'})]
